In [127]:
import pandas as pd
import numpy as np
import math

#open output.csv
pd.set_option('display.float_format', lambda x: '%.f' % x)
path='/Users/sidsharma/Downloads/Output (2).csv'
output = pd.read_csv(path)

#take out processed incompletes
output = output[output['Highest Standard'] != 'Incomplete Information']


#find ratio of 9-12 to total students at each high school
def high_school(df):
    try:
        ratio = float(df['Grades 9-12 Students [Public School] 2016-17'])/float(df['Total Students All Grades (Excludes AE) [Public School] 2016-17'])
        return ratio
    except ValueError: 
        return 0
    
ELSI['9-12 ratio'] = ELSI[['Grades 9-12 Students [Public School] 2016-17', 'Total Students All Grades (Excludes AE) [Public School] 2016-17']].apply(high_school, axis = 1)


#remove all incomplete schools that aren't 100% 9-12
ELSI_copy = ELSI
ELSI = ELSI[ELSI['9-12 ratio'] == 1]

#clean NCES ID columns to transform into floats
def replace_cols(item):
    try:
        if item.startswith('="'):
            return item[2:-1]
    except AttributeError:
        return item
    return item

cols = ELSI.columns
ELSI[cols] = ELSI[cols].applymap(replace_cols)
ELSI['School ID - NCES Assigned [Public School] Latest available year']=ELSI['School ID - NCES Assigned [Public School] Latest available year'].astype('float64')

#drop and append ELSI data
NCES_list = output['NCES School ID'].tolist()
NCES_list = pd.unique(NCES_list).tolist()
ELSI.set_index('School ID - NCES Assigned [Public School] Latest available year', inplace = True)
ELSI.drop(NCES_list, errors ='ignore', inplace = True)
ELSI.reset_index(inplace = True)

def clean(item):
    temp = item.find('[Public School]')
    if temp != -1:
        return item[:temp-1]
    else:
        return item
    
ELSI['Final Standard'] = 'Incomplete Information'
ELSI.reset_index(inplace = True)
ELSI.rename(columns = clean, inplace = True)
output.rename(columns = clean, inplace = True)
ELSI.rename(columns = {'School Name': 'School Name_x', 'School ID - NCES Assigned': 'NCES School ID', 'Web Site URL': 'Link to School Website'}, inplace = True)
output = output.append(ELSI)

path2 = '/Users/sidsharma/Downloads/testadvocacymerge1 - testadvocacymerge1 (12).csv'
advocacy = pd.read_csv(path2)
public = advocacy[advocacy['School Status'] == 'Public']
public.drop(columns = ['Address', 'City', 'State', 'Zip Code'], inplace = True)
private = advocacy[advocacy['School Status'] == 'Private']

#take advocacy sheet and merge with output
output = output.merge(public, left_on = 'NCES School ID', right_on = 'School ID - NCES Assigned [Public School] Latest available year', how = 'left')


#change 'Standard' column to Incomplete Information
def fill(item):
    try:
        if math.isnan(item):
            return "Incomplete Information"            
    except TypeError:
        return item
    return item
output['Standard'] = output['Standard'].apply(fill)

#change standards columns to numbers
def standard_class(item):
    if item == 'Gold Standard':
        return 3
    elif item == 'Silver Standard':
        return 2
    elif item == 'Bronze Standard':
        return 1
    else:
        return 0
output['Standard'] = output['Standard'].apply(standard_class) 
output['Highest Standard']= output['Highest Standard'].apply(standard_class)

#compare two columns
def standard_compare(df):
    if df['Standard'] != 0:
        return df['Standard']
    else:
        return df['Highest Standard']
    
output['Final Standard'] = output[['School Name_x_x', 'Highest Standard', 'Standard']].apply(standard_compare, axis = 1)

#convert back 
def standard_class_2(item):
    if item == 3:
        return "Gold Standard"
    elif item == 2:
        return 'Silver Standard'
    elif item == 1:
        return "Bronze Standard"
    else:
        return "Incomplete Information"
    
output['Final Standard'] = output['Final Standard'].apply(standard_class_2)

def course_compare(df):
    try:
        if math.isnan(df['Class Name_x']):
            return df['Proposed Course Name']
    except(TypeError):
        return df['Class Name_x']
    return df['Class Name_x']
    
output['Final Class Name'] = output[['Class Name_x', 'Proposed Course Name']].apply(course_compare, axis = 1)

#merge other stats forgotten in original iteration
output.drop(columns = ['Free and Reduced Lunch Students'], inplace = True)
ELSI_keep = ['NCES School ID', 'Free and Reduced Lunch Students']
ELSI_copy.rename(columns = {'School ID - NCES Assigned [Public School] Latest available year': 'NCES School ID', 'Free and Reduced Lunch Students [Public School] 2016-17': 'Free and Reduced Lunch Students'}, inplace = True)
ELSI_copy = ELSI_copy[ELSI_keep]
cols = ELSI_copy.columns
ELSI_copy[cols] = ELSI_copy[cols].applymap(replace_cols)
ELSI_copy['NCES School ID'] = ELSI_copy['NCES School ID'].astype('float64')

output = output.merge(ELSI_copy, left_on = 'NCES School ID', right_on = 'NCES School ID', how = 'left')

#take remaining private sheet, create into dataframe that matches output, and then append
private.rename(columns = ({'School Name_x': 'School Name_x_x', 'Standard': 'Final Standard', 'School ID - NCES Assigned [Public School] Latest available year':'NCES School ID', 
                           'Address': 'Location Address 1', 'City': 'Location City', 'State': 'Location State Abbr', 'Zip Code': 'Location ZIP'}), inplace = True)
output = output.append(private)
output.reset_index(inplace = True)


#ELSI read in and clean
keep = ['NCES School ID','School Name_x_x','Description of course_x', 'Final Standard', 
      'Full-Time Equivalent (FTE) Teachers', 'Grades 9-12 Students', 'Location Address 1', 'Location City', 'Location State Abbr', 'Location ZIP', 'County Name', 'Latitude', 'Longitude', 'Year of Course Catalog', 
      'Link to School Website', 'Link to Course Catalog_x', 'Free and Reduced Lunch Students', 'Urban-centric Locale', 'Final Class Name']
output = output[keep]
output.rename(columns = {'Final Class Name': 'Class Name_x'}, inplace = True)
output['School Name_x_x'] = output['School Name_x_x'].apply(lambda x: x.title() if isinstance(x, str) else x)
output['Location City'] = output['Location City'].apply(lambda x: x.title() if isinstance(x, str) else x)
output.sort_values(by = ['School Name_x_x'], kind = 'quicksort', inplace = True)
output.drop_duplicates(subset = ['NCES School ID', 'School Name_x_x','Class Name_x'], keep = 'first', inplace = True)
output['Free and Reduced Lunch Students'] = pd.to_numeric(output['Free and Reduced Lunch Students'], errors='coerce')
path2='/Users/sidsharma/Downloads/output full 2018.csv'
output.to_csv(path2)
print('First part done')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


36531448.0


In [129]:
#aggregate course names, and use that one for map 
from functools import reduce
import re
import math
import random
import time


def testaddcomma(series):
    return reduce(lambda x, y: str(x) + ', ' + str(y), series)

state_abbrev = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

aggregate = output.groupby(['NCES School ID','School Name_x_x']).agg({'Final Standard': lambda x: x.iloc[0],
                                                                      'Class Name_x': testaddcomma, 
                                                                      'Grades 9-12 Students': lambda x: x.iloc[0],
                                                                      'Full-Time Equivalent (FTE) Teachers': lambda x: x.iloc[0],
                                                                      'Location Address 1': lambda x: x.iloc[0],
                                                                      'Location City': lambda x: x.iloc[0],
                                                                     'Location State Abbr': lambda x: x.iloc[0],
                                                                     'Location ZIP': lambda x: x.iloc[0],
                                                                     'County Name': lambda x: x.iloc[0],
                                                                     'Latitude': lambda x: x.iloc[0],
                                                                     'Longitude': lambda x: x.iloc[0],
                                                                     'Link to School Website': lambda x: x.iloc[0],
                                                                     'Link to Course Catalog_x': lambda x: x.iloc[0], 
                                                                     'Urban-centric Locale': lambda x: x.iloc[0],
                                                                     'Free and Reduced Lunch Students': lambda x: x.iloc[0]})

#vermont_test = aggregate[aggregate['Location State Abbr'] == 'VT']
aggregate['Location State Abbr'] = aggregate['Location State Abbr'].apply(lambda x: x.strip() if isinstance(x, str) else x)
print(aggregate['Free and Reduced Lunch Students'].sum())



aggregate_gold = aggregate[aggregate['Final Standard'] == 'Gold Standard']
aggregate_gold.reset_index(inplace = True)
print(aggregate_gold['Free and Reduced Lunch Students'].sum())

def distance(lat1, lon1, lat2, lon2):
    radius = 6371 # km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c
    return d

def test(df):
    closest = 100000000
    closest2 = 100000000
    closest3 = 100000000
    dis = 0
    school = ''
    lat1 = df['Latitude']
    lon1 = df['Longitude']
    for x in range(0, len(aggregate_gold) - 1):
        lat2 = aggregate_gold['Latitude'].iloc[x]
        lon2 = aggregate_gold['Longitude'].iloc[x]
        diff_lat = float(lat2) - float(lat1)
        diff_lon = float(lon2) - float(lon1)
        if (abs(diff_lat)> 2) or (abs(diff_lon) >2):
            continue
        dis = distance(float(lat1), float(lon1), float(lat2), float(lon2))
        if (dis < closest) and (dis != 0):
            closest = dis
            df['Closest Gold Standard'] = aggregate_gold['School Name_x_x'].iloc[x]
            df['Closest Gold Standard Location'] = aggregate_gold['Location City'].iloc[x] + ', ' + aggregate_gold['Location State Abbr'].iloc[x]
            df['Closest Distance'] = closest
    return df
        

'''start = time.time()
vermont_test = vermont_test.apply(test, axis = 1)
print('length', len(vermont_test))
print(vermont_test)
end = time.time()
print(end-start)
'''


path3='/Users/sidsharma/Downloads/Aggregate Full List.csv'
print('Full List saved')
aggregate.to_csv(path3)

aggregate['Grades 9-12 Students'] = aggregate['Grades 9-12 Students'].astype('float64')

incomplete_removed = aggregate[aggregate['Final Standard'] != 'Incomplete Information']
path4 = '/Users/sidsharma/Downloads/Tableau Map Aggregate.csv'
aggregate.to_csv(path4)

df1 = pd.DataFrame()

for i in state_abbrev:
    state_temp = incomplete_removed[incomplete_removed['Location State Abbr'] == i]
    gold_temp = state_temp[state_temp['Final Standard'] == 'Gold Standard']
    silver_temp = state_temp[state_temp['Final Standard'] == 'Silver Standard']
    bronze_temp = state_temp[state_temp['Final Standard'] == 'Bronze Standard']
    if len(state_temp) != 0:
        df2 = pd.DataFrame(({'State': [state_abbrev[i]], 'Covered Students': state_temp['Grades 9-12 Students'].sum(),
                            'Gold Standard Students': [gold_temp['Grades 9-12 Students'].sum()],
                            'Silver Standard Students': [silver_temp['Grades 9-12 Students'].sum()],
                            'Bronze Standard Students': [bronze_temp['Grades 9-12 Students'].sum()],
                            'Gold Standard Schools': [len(gold_temp)],
                            'Silver Standard Schools': [len(silver_temp)],
                            'Bronze Standard Schools': [len(bronze_temp)]}))
        df1 = df1.append(df2)
    '''print((state_abbrev[i], state_temp['Grades 9-12 Students'].sum(),
          gold_temp['Grades 9-12 Students'].sum(),
          silver_temp['Grades 9-12 Students'].sum(),
          bronze_temp['Grades 9-12 Students'].sum(),
          len(gold_temp), 
          len(silver_temp),
          len(bronze_temp),
        ))'''
df1.reset_index(inplace = True)
df1.drop(columns = 'index', inplace = True)

path5 = '/Users/sidsharma/Downloads/Summary Stats updated.csv'
df1.to_csv(path5)
print("Summary Stats updated")


path4='/Users/sidsharma/Downloads/people-1732955-589.csv'
persons = pd.read_csv(path4)
#persons['Organization - NCES ID'] = persons['Organization - NCES ID'].apply(lambda x: x if x.isalnum() else 'No NCES ID')

def findsplit(item):
    try:
        if isinstance(item, str):
            if item.find(',') != -1:
                list = item.split(',')
                return float(list[0])
            item = re.sub("[^0-9]", "", item)
            return float(item[:12])
    except(ValueError):
        if isinstance(item, float) or isinstance(item, int):
            return item
        else:
            return -1

def emailsplit(item):
    if isinstance(item, str):
        if item.find(',') != -1:
            list = item.split(',')
            return list[0]
        else:
            return item
    else: 
        return 'No Email'
    
persons['Organization - NCES ID'] = persons['Organization - NCES ID'].apply(findsplit)
persons['Organization - NCES ID'] = persons['Organization - NCES ID'].astype('float64', errors = 'raise')
persons['Person - Email'] = persons['Person - Email'].apply(emailsplit)
#persons = persons[persons['Person - Email'] != 'No Email']
aggregate.replace({'Location State Abbr': state_abbrev}, inplace = True)
persons = persons.merge(aggregate, left_on = 'Organization - NCES ID', right_on = 'NCES School ID', how = 'inner')
keep = ['Person - Name', 'Person - Organization', 'Person - Email', 'Organization - NCES ID', 'Grades 9-12 Students', 'Location Address 1', 'Location City', 'Location State Abbr',
       'Location ZIP', 'County Name', 'Link to School Website']
persons = persons[keep]

#read in aggregate for states

state = pd.read_csv('50 States Summary Statistics (2018-2019) - Sheet1.csv')
state.rename(columns = {'Unnamed: 0': 'State Name'}, inplace = True)
us_rate = (state['Total Gold Standard Students (9-12)'].iloc[50])/(state['Total Checked Students (9-12)'].iloc[50])

def divide_two(df):
    x = df['Total Gold Standard Students (9-12)']
    y = df['Total Checked Students (9-12)']
    return x/y


state['Gold Standard percent'] = state[['Total Gold Standard Students (9-12)', 'Total Checked Students (9-12)']].apply(divide_two, axis = 1)
state.reset_index(inplace = True)


def standard_message(df):
    if (us_rate < df['Gold Standard percent']):
        return 'In other news, ' + df['State Name'] + " performed phenomenally in this year's assessment of personal finance quality with a total of " + str(int(df['Total Gold Standard Students (9-12)'])) + ' Gold Standard students over ' + str(int(df['Gold Standard 2018-19'])) + ' Gold Standard schools, ' + str(int(df['Silver Standard 2018-19'])) + ' Silver Standard schools, and ' + str(int(df['Bronze 2018-19'])) + " Bronze Standard schools. It is teachers like you that have helped your school educate so many schools in personal finance. Congratulations!"
    else:
        return (df['State Name'] + ' finished the year with ' + str(int(df['Gold Standard 2018-19'])) + ' Gold Standard schools, '+ str(int(df['Silver Standard 2018-19']))
                 + ' Silver Standard schools, and ' + str(int(df['Bronze 2018-19'])) + ' Bronze Standard schools. In total, ' + str(int(df['Total Covered Students(9-12)'])) + ' were covered in your state.')

state = state.truncate(after = 50)

state['Standard Message'] = state[['Possible Rank', 'Total Covered Students(9-12)', 'State Name', 'Gold Standard percent', 'Total Gold Standard Students (9-12)', 'Gold Standard 2018-19', 'Silver Standard 2018-19', 'Bronze 2018-19']].apply(standard_message, axis =1)

email_list = persons.merge(state, left_on = 'Location State Abbr', right_on = 'State Name', how = 'left')
keep = ['Person - Name', 'Person - Email', 'Person - Organization',
       'Organization - NCES ID', 'Grades 9-12 Students',
       'Location City', 'Location State Abbr',
       'Link to School Website', 'Total Checked Students (9-12)', 'Total Students (9-12)',
       'Total Covered Students(9-12)', 'Total Gold Standard Students (9-12)',
       'Gold Standard Charters', 'Gold Standard 2018-19', 'Silver Standard 2018-19', 
       'Bronze 2018-19','Non-existent personal finance course or incomplete information 2018-19',
       'Point Average 2018-2019', 'Possible Rank', 'Gold Standard percent',
       'Standard Message']
email_list = email_list[keep]
path5 = '/Users/sidsharma/Downloads/50 States Email List.csv'
email_list.to_csv(path5)

email_list.drop_duplicates(subset = ['Organization - NCES ID', 'Person - Organization'], keep = 'first', inplace = True)
#print(email_list['Grades 9-12 Students'].sum()) 

#determine number of non-required gold standard schools
required_states = {'Alabama': 'N', 'Wyoming': 'N', 'Tennessee': 'N', 'Utah': 'N', 'Missouri': 'N', 'Virginia': 'N', '': 'N'}
aggregate.replace({'Location State Abbr': required_states}, inplace = True)
aggregate = aggregate[aggregate['Final Standard'] == 'Gold Standard']
aggregate = aggregate[aggregate['Location State Abbr'] != 'N']
print('Number of Gold Standard schools in non-required states:', len(aggregate))

12
5791654.0
513084.0
Full List saved
AK
AL
AR
AS
AZ
CA
CO
CT
DC
DE
FL
GA
GU
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MP
MS
MT
NA
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
PR
RI
SC
SD
TN
TX
UT
VA
VI
VT
WA
WI
WV
WY
Summary Stats updated
16179
7268
Number of Gold Standard schools in non-required states: 758
